# Yelp and Simple MLP

### Read in CSV

In [38]:
import pandas as pd
import numpy as np
import keras
import tensorflow as tf

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.utils import np_utils

In [39]:
session_conf = tf.ConfigProto(
      intra_op_parallelism_threads=1,
      inter_op_parallelism_threads=1)
sess = tf.Session(config=session_conf)

In [40]:
df = pd.read_csv('HOPEFUL.csv')

In [41]:
df = df.drop(df.columns[df.columns.str.contains('unnamed', case = False)],axis = 1)
df = df.drop(['business_id', 'date', 'user_id', 'user_cool', 'user_funny', 'user_userful', 'business_city', 'business_zip'], axis=1)
df.head(15)

,stars,name_of_month,day_of_week,user_average_stars,user_num_reviews,user_num_friends,business_num_reviews,business_average_stars,business_catgrylv2,business_catgrylv1,business_lat,business_long,lv1Cat_Av,lv2Cat_Av,zipCodeBusiness_Av,zipCodeUser_Av,zipCodeUser_WeightedAv,zipCodeUserCount
0,5,May,Saturday,4.67,6,1,1953,4.0,Bars,Breakfast & Brunch,45.516373,-73.577537,3.739232,3.731733,3.980257,3.779459,3.720424,2162
1,5,May,Saturday,4.67,6,1,84,4.0,Bars,Food,45.523333,-73.594859,3.719693,3.731733,4.031579,3.875368,3.711489,95
2,5,May,Saturday,4.67,6,1,50,4.5,Bars,Breakfast & Brunch,45.472902,-73.588321,3.739232,3.731733,4.280899,3.843488,3.927304,86
3,5,May,Saturday,4.67,6,1,70,4.0,Bars,Breakfast & Brunch,45.522144,-73.607076,3.739232,3.731733,3.943038,3.837975,3.851323,79
4,4,May,Saturday,4.67,6,1,61,3.5,Bars,Breakfast & Brunch,45.502510,-73.570119,3.739232,3.731733,3.119792,3.609978,3.655213,462
5,4,May,Saturday,4.67,6,1,397,4.5,Bars,Bars,45.503544,-73.554469,3.774976,3.731733,4.289264,3.822211,3.788127,493
6,5,September,Wednesday,5.00,1,0,38,4.5,Active Life,Food,36.171869,-115.142146,3.719693,3.504678,3.837694,3.798199,3.771875,46843
7,4,May,Friday,3.62,359,48,111,4.0,Bars,Steakhouses,43.696175,-79.329092,3.802124,3.731733,3.996310,3.620204,3.503142,245
8,4,October,Tuesday,3.62,359,48,64,4.0,Restaurants,Restaurants,43.678484,-79.358255,3.605285,3.677544,4.164706,3.800964,3.530308,83
9,3,February,Saturday,3.62,359,48,251,3.5,Restaurants,Restaurants,43.643497,-79.423201,3.605285,3.677544,3.539823,3.668795,3.673870,531


In [42]:
def myFunc(starValue):
    if starValue == 5:
        newStarValue = 1
    elif starValue == 4:
        newStarValue = 1
    elif starValue == 3:
        newStarValue = 0
    elif starValue == 2:
        newStarValue = 0
    elif starValue == 1:
        newStarValue = 0
    return newStarValue

df['stars'] = df['stars'].apply(myFunc)
df.head(3)

,stars,name_of_month,day_of_week,user_average_stars,user_num_reviews,user_num_friends,business_num_reviews,business_average_stars,business_catgrylv2,business_catgrylv1,business_lat,business_long,lv1Cat_Av,lv2Cat_Av,zipCodeBusiness_Av,zipCodeUser_Av,zipCodeUser_WeightedAv,zipCodeUserCount
0,1,May,Saturday,4.67,6,1,1953,4.0,Bars,Breakfast & Brunch,45.516373,-73.577537,3.739232,3.731733,3.980257,3.779459,3.720424,2162
1,1,May,Saturday,4.67,6,1,84,4.0,Bars,Food,45.523333,-73.594859,3.719693,3.731733,4.031579,3.875368,3.711489,95
2,1,May,Saturday,4.67,6,1,50,4.5,Bars,Breakfast & Brunch,45.472902,-73.588321,3.739232,3.731733,4.280899,3.843488,3.927304,86


In [43]:
# Label Encode Categorical Data
# name_of_month
df['name_of_month'] = df['name_of_month'].astype('category')
df['name_of_month'] = df['name_of_month'].cat.codes

# day_of_week
df['day_of_week'] = df['day_of_week'].astype('category')
df['day_of_week'] = df['day_of_week'].cat.codes

# business_catgrylv2
df['business_catgrylv2'] = df['business_catgrylv2'].astype('category')
df['business_catgrylv2'] = df['business_catgrylv2'].cat.codes

# business_catgrylv1
df['business_catgrylv1'] = df['business_catgrylv1'].astype('category')
df['business_catgrylv1'] = df['business_catgrylv1'].cat.codes

# Check Categorization
df.head(3)

,stars,name_of_month,day_of_week,user_average_stars,user_num_reviews,user_num_friends,business_num_reviews,business_average_stars,business_catgrylv2,business_catgrylv1,business_lat,business_long,lv1Cat_Av,lv2Cat_Av,zipCodeBusiness_Av,zipCodeUser_Av,zipCodeUser_WeightedAv,zipCodeUserCount
0,1,8,2,4.67,6,1,1953,4.0,1,9,45.516373,-73.577537,3.739232,3.731733,3.980257,3.779459,3.720424,2162
1,1,8,2,4.67,6,1,84,4.0,1,22,45.523333,-73.594859,3.719693,3.731733,4.031579,3.875368,3.711489,95
2,1,8,2,4.67,6,1,50,4.5,1,9,45.472902,-73.588321,3.739232,3.731733,4.280899,3.843488,3.927304,86


In [44]:
# Normalize
df = (df - df.mean()) / (df.max() - df.min())
df.head(3)

,stars,name_of_month,day_of_week,user_average_stars,user_num_reviews,user_num_friends,business_num_reviews,business_average_stars,business_catgrylv2,business_catgrylv1,business_lat,business_long,lv1Cat_Av,lv2Cat_Av,zipCodeBusiness_Av,zipCodeUser_Av,zipCodeUser_WeightedAv,zipCodeUserCount
0,0.339019,0.223576,-0.166741,0.233088,-0.009673,-0.007536,0.217769,0.067103,-0.261601,-0.301905,0.065042,0.105853,0.004180,0.000537,0.062301,0.011980,-0.006591,-0.152566
1,0.339019,0.223576,-0.166741,0.233088,-0.009673,-0.007536,-0.036240,0.067103,-0.261601,-0.051905,0.065097,0.105786,-0.006062,0.000537,0.075132,0.035957,-0.008825,-0.161757
2,0.339019,0.223576,-0.166741,0.233088,-0.009673,-0.007536,-0.040861,0.192103,-0.261601,-0.301905,0.064697,0.105812,0.004180,0.000537,0.137462,0.027987,0.045128,-0.161797


### Split into Train and Test Data

In [45]:
from sklearn.model_selection import train_test_split

In [46]:
y = df.pop('stars')
X = df

In [47]:
X_train, X_test, y_train, y_test = train_test_split(X.index,y,test_size=0.20)

In [48]:
print(X_train.shape)
print(X_test.shape)

(4202787,)
(1050697,)


In [49]:
print(y_train.shape)
print(y_test.shape)

(4202787,)
(1050697,)


In [50]:
# We need to get y sets to one-hot-encoded values
def create_one_hot_encoded_array(array):
    uniques, ids = np.unique(array, return_inverse=True)
    return np_utils.to_categorical(ids, len(uniques))

y_train = create_one_hot_encoded_array(y_train)
y_test = create_one_hot_encoded_array(y_test)

print(y_train[:2])

[[1. 0.]
 [1. 0.]]


In [51]:
print(X_train[:2])

Int64Index([1762167, 3207133], dtype='int64')


In [52]:
X_train = X.iloc[X_train]

In [53]:
print(X_train[:2])

         name_of_month  day_of_week  user_average_stars  user_num_reviews  \
1762167      -0.321879     0.499926            0.035588          0.004297   
3207133      -0.140060    -0.500074           -0.256912         -0.006829   

         user_num_friends  business_num_reviews  business_average_stars  \
1762167         -0.007603              0.218856               -0.182897   
3207133         -0.007603             -0.045754               -0.182897   

         business_catgrylv2  business_catgrylv1  business_lat  business_long  \
1762167           -0.261601            0.371172     -0.009584      -0.055667   
3207133           -0.261601            0.159633     -0.017055       0.077501   

         lv1Cat_Av  lv2Cat_Av  zipCodeBusiness_Av  zipCodeUser_Av  \
1762167  -0.024896   0.000537           -0.028613       -0.013003   
3207133  -0.000871   0.000537           -0.019620       -0.016373   

         zipCodeUser_WeightedAv  zipCodeUserCount  
1762167                0.001422          

In [54]:
print(type(X_train))

<class 'pandas.core.frame.DataFrame'>


In [55]:
print(X_train.shape)

(4202787, 17)


In [56]:
print(y_train.shape)

(4202787, 2)


In [57]:
X_test = X.iloc[X_test]

In [58]:
print(X_test.shape)

(1050697, 17)


In [59]:
print(y_test.shape)

(1050697, 2)


### Define Network

In [60]:
model = Sequential()

In [61]:
model.add(Dense(100, input_shape=(17,)))

In [62]:
model.add(Activation('relu'))

In [63]:
model.add(Dropout(0.05))

In [64]:
model.add(Dense(100))

In [65]:
model.add(Activation('relu'))

In [66]:
model.add(Dropout(0.05))

In [67]:
model.add(Dense(50))

In [68]:
model.add(Activation('relu'))

In [69]:
model.add(Dropout(0.05))

In [70]:
model.add(Dense(2))

In [71]:
model.add(Activation('softmax'))

### Compile Network

In [72]:
from keras import optimizers
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['accuracy'])

### Fit Network

In [73]:
history = model.fit(X_train, y_train, 
          batch_size=50, epochs=1, verbose=1,
          validation_data=(X_test, y_test)
         )

Train on 4202787 samples, validate on 1050697 samples
Epoch 1/1
4202787/4202787 [==============================] - 248s 59us/step - loss: 0.4534 - acc: 0.7762 - val_loss: 0.4499 - val_acc: 0.7774


In [74]:
np.unique(model.predict_classes(X_train))

array([0, 1])

In [75]:
import matplotlib.pyplot as plt
pd.Series(history.history['loss']).plot(logy=True)
plt.xlabel("Epoch")
plt.ylabel("Train Error")
plt.savefig('Yelp_Simple_MLP_V1')
plt.show()

<Figure size 640x480 with 1 Axes>

In [76]:
from sklearn.metrics import classification_report

Y_test = np.argmax(y_test, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_test)
print(classification_report(Y_test, y_pred))

Y_train = np.argmax(y_train, axis=1) # Convert one-hot to index
y_pred = model.predict_classes(X_train)
print(classification_report(Y_train, y_pred))

             precision    recall  f1-score   support

          0       0.72      0.57      0.63    356051
          1       0.80      0.88      0.84    694646

avg / total       0.77      0.78      0.77   1050697

             precision    recall  f1-score   support

          0       0.71      0.57      0.63   1424978
          1       0.80      0.88      0.84   2777809

avg / total       0.77      0.78      0.77   4202787

